In [ ]:
from bisect import bisect


# Getting the data

In [ ]:
# setting the parameters
NVALID = 0.1  # size of validation set


# Splitting

In [ ]:
splitter = GroupShuffleSplit(n_splits=1, test_size=NVALID, random_state=0)

ids = df.index.unique('id')  # get all the unique ids
ancestors = get_ancestors(data_dir, ids)  # find ancestor by id if it exists
# split the ids using groups. This way the same group/notebooks will be in the test or in the training
ids_train, ids_valid = next(splitter.split(ids, groups=ancestors)) 
ids_train, ids_valid = ids[ids_train], ids[ids_valid]

df_train = df.loc[ids_train, :]
df_valid = df.loc[ids_valid, :]

print(f"Shape of train: {df_train.shape[0]}; validation: {df_valid.shape[0]}")

In [ ]:
y_train = df_ranks.loc[ids_train].to_numpy()  # get all required train results
groups = df_ranks.loc[ids_train].groupby('id').size().to_numpy() # Number of cells in each notebook. will later be used to help xgboost make a ranking

In [ ]:
y_valid = df_orders.loc[ids_valid]

In [ ]:
y_pred = pd.DataFrame({'rank': model.predict(X_valid)}, index=df_valid.index)
y_pred = (
    y_pred
    .sort_values(['id', 'rank'])  # Sort the cells in each notebook by their rank.
                                  # The cell_ids are now in the order the model predicted.
    .reset_index('cell_id')  # Convert the cell_id index into a column.
    .groupby('id')['cell_id'].apply(list)  # Group the cell_ids for each notebook into a list.
)

# Metrics 

In [2]:
def count_inversions(a):
    inversions = 0
    sorted_so_far = []
    for i, u in enumerate(a):
        j = bisect(sorted_so_far, u)
        inversions += i - j
        sorted_so_far.insert(j, u)
    return inversions


def kendall_tau(ground_truth, predictions):
    total_inversions = 0
    total_2max = 0  # twice the maximum possible inversions across all instances
    for gt, pred in zip(ground_truth, predictions):
        ranks = [gt.index(x) for x in pred]  # rank predicted order in terms of ground truth
        total_inversions += count_inversions(ranks)
        n = len(gt)
        total_2max += n * (n - 1)
    return 1 - 4 * total_inversions / total_2max


## Results

In [ ]:
y_dummy = df_valid.reset_index('cell_id').groupby('id')['cell_id'].apply(list)
kendall_tau(y_valid, y_dummy)

In [ ]:
kendall_tau(y_valid, y_pred)

## Looking at the data

In [ ]:
nb_id = df_valid.index.get_level_values('id').unique()[8]

display(df.loc[nb_id].loc[y_valid.loc[nb_id]][['source', 'cell_type']])
display(df.loc[nb_id].loc[y_pred.loc[nb_id]][['source', 'cell_type']])